<a href="https://colab.research.google.com/github/ZainaHweij/csdiagnostic/blob/main/linear_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linear Algorithm

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Import testing data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/content/drive/MyDrive/SEF 2023-2024/TinyML/training_three.csv')

# Split data into features (X) and target (y)
X = df[['fsr_1', 'fsr_2','fsr_3','fsr_4','fsr_5']]
y = df['ground_truth']

df.head()
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

Generate testing data

In [ ]:
import statsmodels.api as sm


X = df[['fsr_1', 'fsr_2', 'fsr_3', 'fsr_4', 'fsr_5']]
X = sm.add_constant(X)  # Add a constant term for the intercepts

y = df['ground_truth']

model = sm.OLS(y, X).fit()
print(model.summary()) # generate report of stats between data

In [ ]:
# Assuming you have loaded your data into the DataFrame 'df'
# Assuming you have the linear regression coefficients available
coefficients = {'const': 0.3120, 'fsr_1': -0.0007, 'fsr_2': -0.0009, 'fsr_3': -0.0052, 'fsr_4': 0.0048, 'fsr_5': 0.0066}

# Calculate predictions using the specified equation
predictions = (
    coefficients['const'] +
    coefficients['fsr_1'] * df['fsr_1'] +
    coefficients['fsr_2'] * df['fsr_2'] +
    coefficients['fsr_3'] * df['fsr_3'] +
    coefficients['fsr_4'] * df['fsr_4'] +
    coefficients['fsr_5'] * df['fsr_5']
)

# Apply the threshold
threshold = 30
binary_predictions = predictions > threshold

# Add predictions to the DataFrame
df['predictions'] = binary_predictions.astype(int)

# Display the DataFrame with predictions
print(df[['fsr_1', 'fsr_2', 'fsr_3', 'fsr_4', 'fsr_5', 'ground_truth', 'predictions']])

# Calculate accuracy
accuracy = sum(df['predictions'] == df['ground_truth']) / len(df)

# Calculate precision
true_positives = sum((df['predictions'] == 1) & (df['ground_truth'] == 1))
false_positives = sum((df['predictions'] == 1) & (df['ground_truth'] == 0))
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

# Display results
print(f"Accuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")

Calculate mmHG from voltage readings

In [ ]:
# Calculate the average FSR reading from the DataFrame
df['fsr_1'] = (df['fsr_1'] / 1024) * 3.3
df['fsr_2'] = (df['fsr_2'] / 1024) * 3.3
df['fsr_3'] = (df['fsr_3'] / 1024) * 3.3
df['fsr_4'] = (df['fsr_4'] / 1024) * 3.3
df['fsr_5'] = (df['fsr_5'] / 1024) * 3.3

average_fsr_reading = (df['fsr_1'] + df['fsr_2'] + df['fsr_3'] + df['fsr_4'] + df['fsr_5']) / 5

# Calculate FSR resistance in ohm based on the Arduino code logic
fsr_resistance = ((5000 - average_fsr_reading) * 10000) / average_fsr_reading

# Calculate FSR conductance based on the Arduino code logic
fsr_conductance = 1000000 / fsr_resistance

if (fsr_conductance.all() <= 1000):
  fsr_force = fsr_conductance / 80
else:
  fsr_force = (fsr_conductance - 1000) / 30

# Convert conductance to mmHG
fsr_mmHG = 0.0075006157584566 * fsr_force

# Use threshold values to create binary predictions
threshold = 30
binary_predictions = threshold > threshold

# Add predictions to the DataFrame
df['predictions'] = binary_predictions.astype(int)

# Display the DataFrame with predictions
print(df[['fsr_1', 'fsr_2', 'fsr_3', 'fsr_4', 'fsr_5', 'ground_truth', 'predictions']])

# Calculate accuracy
accuracy = sum(df['predictions'] == df['ground_truth']) / len(df)

# Calculate precision
true_positives = sum((df['predictions'] == 1) & (df['ground_truth'] == 1))
false_positives = sum((df['predictions'] == 1) & (df['ground_truth'] == 0))
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

# Display results
print(f"Accuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(fsr_mmHG)